<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_14_04_ids_kdd99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading the csv

In [1]:
import pandas as pd
from tensorflow.keras.utils import get_file

pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)

try:
    p = get_file('kdd-with-columns.csv', origin=\
    'https://github.com/jeffheaton/jheaton-ds2/raw/main/'\
    'kdd-with-columns.csv',archive_format=None)
except:
    print('Error downloading')
    raise
    
print(p) 


dataframe = pd.read_csv(p)

print("Read {} rows.".format(len(dataframe)))

dataframe.dropna(inplace=True,axis=1) 

pd.set_option('display.max_columns', 5)
pd.set_option('display.max_rows', 5)
dataframe

/Users/yusufsalem/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


/Users/yusufsalem/.keras/datasets/kdd-with-columns.csv
Read 494021 rows.


,duration,protocol_type,...,dst_host_srv_rerror_rate,outcome
0,0,tcp,...,0.0,normal.
1,0,tcp,...,0.0,normal.
...,...,...,...,...,...
494019,0,tcp,...,0.0,normal.
494020,0,tcp,...,0.0,normal.


## Analyze kdd



In [2]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(dataframe):
    print()
    columns = dataframe.columns.values
    sum = float(len(dataframe))

    print("{} rows".format(int(sum)))
    for col in columns:
        unique = dataframe[col].unique()
        unumber = len(unique)
        if unumber>100:
            print("** {}:{} ({}%)".format(col,unumber,\
                int(((unumber)/sum)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(dataframe[col])))
            expand_categories(dataframe[col])

The analyzation part

In [3]:

analyze(dataframe)


494021 rows
** duration:2495 (0%)
** protocol_type:[icmp:57.41%,tcp:38.47%,udp:4.12%]
** service:[ecr_i:56.96%,private:22.45%,http:13.01%,smtp:1.97%,other:1.46%,domain_u:1.19%,ftp_data:0.96%,eco_i:0.33%,ftp:0.16%,finger:0.14%,urp_i:0.11%,telnet:0.1%,ntp_u:0.08%,auth:0.07%,pop_3:0.04%,time:0.03%,csnet_ns:0.03%,remote_job:0.02%,gopher:0.02%,imap4:0.02%,discard:0.02%,domain:0.02%,iso_tsap:0.02%,systat:0.02%,shell:0.02%,echo:0.02%,rje:0.02%,whois:0.02%,sql_net:0.02%,printer:0.02%,nntp:0.02%,courier:0.02%,sunrpc:0.02%,netbios_ssn:0.02%,mtp:0.02%,vmnet:0.02%,uucp_path:0.02%,uucp:0.02%,klogin:0.02%,bgp:0.02%,ssh:0.02%,supdup:0.02%,nnsp:0.02%,login:0.02%,hostnames:0.02%,efs:0.02%,daytime:0.02%,link:0.02%,netbios_ns:0.02%,pop_2:0.02%,ldap:0.02%,netbios_dgm:0.02%,exec:0.02%,http_443:0.02%,kshell:0.02%,name:0.02%,ctf:0.02%,netstat:0.02%,Z39_50:0.02%,IRC:0.01%,urh_i:0.0%,X11:0.0%,tim_i:0.0%,pm_dump:0.0%,tftp_u:0.0%,red_i:0.0%]
** flag:[SF:76.6%,S0:17.61%,REJ:5.44%,RSTR:0.18%,RSTO:0.12%,SH:0.02%,S

## Encode the features

In [4]:
# Encode a numeric column as zscores
def encode_numeric_zscore(dataframe, name, mean=None, m1=None):
    if mean is None:
        mean = dataframe[name].mean()

    if m1 is None:
        m1 = dataframe[name].std()

    dataframe[name] = (dataframe[name] - mean) / m1
    
# Encode text values to dummy variables(i.e. [1,0,0],
# [0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(dataframe, name):
    dummies = pd.get_dummies(dataframe[name])
    for x in dummies.columns:
        dname = f"{name}-{x}"
        dataframe[dname] = dummies[x]
    dataframe.drop(name, axis=1, inplace=True)


Conversion part

In [5]:
# encoding part
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)

for name in dataframe.columns:
  if name == 'outcome':
    pass
  elif name in ['protocol_type','service','flag','land','logged_in',
                'is_host_login','is_guest_login']:
    encode_text_dummy(dataframe,name)
  else:
    encode_numeric_zscore(dataframe,name)    

# Only first 5

dataframe.dropna(inplace=True,axis=1)
dataframe[0:5]


# Classify
xcol = dataframe.columns.drop('outcome')
x = dataframe[xcol].values
dummies = pd.get_dummies(dataframe['outcome']) # Classification
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values

/var/folders/b9/d2jv81ks3h1_7p8zz32hz5740000gn/T/ipykernel_26101/1362819603.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataframe[dname] = dummies[x]
/var/folders/b9/d2jv81ks3h1_7p8zz32hz5740000gn/T/ipykernel_26101/1362819603.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataframe[dname] = dummies[x]
/var/folders/b9/d2jv81ks3h1_7p8zz32hz5740000gn/T/ipykernel_26101/1362819603.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

Number of attacks

In [6]:
dataframe.groupby('outcome')['outcome'].count()

outcome
back.               2203
buffer_overflow.      30
                    ... 
warezclient.        1020
warezmaster.          20
Name: outcome, Length: 23, dtype: int64